# Question

In [76]:
# How can we balance personalization with cost-effectiveness in marketing campaigns?
# Objective: Design a cost-benefit analysis model to assess the impact of personalized marketing strategies.
# Expected Outcome: Propose a model that ensures a balance between personalization and cost management.


# Importing Packages

In [77]:
import pandas as pd

# Getting Data

In [ ]:
# Get data
campaigns = pd.read_csv("../Data DSA3101/campaigns.csv")
engagement = pd.read_csv("../Data DSA3101/customer_engagement.csv")
customer = pd.read_csv("../Data DSA3101/customers.csv")

print(campaigns.shape)
print(engagement.shape)
print(customer.shape)

(50, 11)
(28921, 11)
(10000, 17)


In [79]:
# Merge data
merged = pd.merge(engagement, campaigns, left_on='campaign_id', right_on='campaign_id', how='inner')
print(merged.shape)

(28921, 21)


In [80]:
merged_customer = pd.merge(merged, customer, left_on="customer_id", right_on="customer_id", how="inner")
print(merged.shape)

(28921, 21)


In [81]:
campaigns

,campaign_id,customer_segment,campaign_name,campaign_type,start_date,end_date,recommended_product_id,recommended_product_name,campaign_budget,total_campaign_cost,total_revenue_generated
0,1,Retired,Wealth Management Campaign for Retired,Email,2022-03-17,2022-05-04,107,Wealth Management,58055.93,70597.33,94151.78
1,2,Middle-Market,Auto Loan Campaign for Middle-Market,Mobile App Notifications,2022-03-23,2022-05-02,106,Auto Loan,35527.89,87461.14,184133.52
2,3,Retired,Mortgage Campaign for Retired,Email,2022-05-22,2022-06-26,103,Mortgage,52445.28,56053.99,160828.61
3,4,Budget-Conscious,Personal Loan Campaign for Budget-Conscious,Email,2023-01-13,2023-02-21,102,Personal Loan,57816.78,22910.89,58342.54
4,5,Young Professionals,Personal Loan Campaign for Young Professionals,Mobile App Notifications,2023-12-06,2024-01-13,102,Personal Loan,36979.25,24374.42,63738.91
5,6,Middle-Market,Savings Account Campaign for Middle-Market,Email,2023-12-25,2024-02-19,104,Savings Account,36112.27,92445.40,149392.76
6,7,Young Professionals,Credit Card Campaign for Young Professionals,SMS,2023-03-18,2023-04-28,101,Credit Card,64751.26,10520.47,19903.98
7,8,Middle-Market,Personal Loan Campaign for Middle-Market,SMS,2024-04-11,2024-05-19,102,Personal Loan,29313.53,47256.53,54272.54
8,9,Young Professionals,Personal Loan Campaign for Young Professionals,SMS,2023-08-07,2023-09-25,102,Personal Loan,73087.05,47119.67,62738.27
9,10,Budget-Conscious,Auto Loan Campaign for Budget-Conscious,Mobile App Notifications,2022-07-02,2022-08-05,106,Auto Loan,67625.11,59348.91,159796.32


In [82]:
engagement.head()

,engagement_id,customer_id,campaign_id,channel,sent,delivered,opened,clicked,engagement_date,engagement_time,conversion_status
0,1,3110,1,Email,Yes,Yes,No,No,2022-03-26 14:34:53,14:34:53,No
1,2,2468,1,Email,Yes,Yes,No,No,2022-03-29 06:49:52,06:49:52,Yes
2,3,7168,1,Email,Yes,Yes,No,No,2022-04-02 13:34:07,13:34:07,No
3,4,9616,1,Email,Yes,Yes,No,No,2022-03-26 11:50:42,11:50:42,No
4,5,7365,1,Email,Yes,Yes,No,Yes,2022-04-05 14:36:54,14:36:54,No


In [83]:
customer.head()

,customer_id,age,job,marital,education,credit_default,customer_segment,region,income,created_at,credit_card,personal_loan,mortgage,savings_account,investment_product,auto_loan,wealth_management
0,1,38,blue-collar,married,secondary,no,High-Value,Urban,2603,2020-09-23 22:11:32.308466,yes,yes,yes,yes,yes,yes,yes
1,2,18,student,divorced,tertiary,no,Young Professionals,Suburban,793,2022-04-25 00:06:57.094379,yes,yes,no,yes,yes,no,no
2,3,73,services,divorced,secondary,no,Budget-Conscious,Urban,3130,2021-07-21 13:09:42.193173,yes,no,no,no,yes,yes,no
3,4,87,retired,married,primary,no,Retired,Urban,2401,2024-04-21 00:54:54.070579,yes,yes,no,yes,no,no,no
4,5,20,technician,single,primary,no,Budget-Conscious,Suburban,3354,2020-06-04 02:12:43.905811,yes,yes,no,yes,no,yes,no


In [84]:
merged.head()

,engagement_id,customer_id,campaign_id,channel,sent,delivered,opened,clicked,engagement_date,engagement_time,...,customer_segment,campaign_name,campaign_type,start_date,end_date,recommended_product_id,recommended_product_name,campaign_budget,total_campaign_cost,total_revenue_generated
0,1,3110,1,Email,Yes,Yes,No,No,2022-03-26 14:34:53,14:34:53,...,Retired,Wealth Management Campaign for Retired,Email,2022-03-17,2022-05-04,107,Wealth Management,58055.93,70597.33,94151.78
1,2,2468,1,Email,Yes,Yes,No,No,2022-03-29 06:49:52,06:49:52,...,Retired,Wealth Management Campaign for Retired,Email,2022-03-17,2022-05-04,107,Wealth Management,58055.93,70597.33,94151.78
2,3,7168,1,Email,Yes,Yes,No,No,2022-04-02 13:34:07,13:34:07,...,Retired,Wealth Management Campaign for Retired,Email,2022-03-17,2022-05-04,107,Wealth Management,58055.93,70597.33,94151.78
3,4,9616,1,Email,Yes,Yes,No,No,2022-03-26 11:50:42,11:50:42,...,Retired,Wealth Management Campaign for Retired,Email,2022-03-17,2022-05-04,107,Wealth Management,58055.93,70597.33,94151.78
4,5,7365,1,Email,Yes,Yes,No,Yes,2022-04-05 14:36:54,14:36:54,...,Retired,Wealth Management Campaign for Retired,Email,2022-03-17,2022-05-04,107,Wealth Management,58055.93,70597.33,94151.78


In [85]:
merged.columns

Index(['engagement_id', 'customer_id', 'campaign_id', 'channel', 'sent',
       'delivered', 'opened', 'clicked', 'engagement_date', 'engagement_time',
       'conversion_status', 'customer_segment', 'campaign_name',
       'campaign_type', 'start_date', 'end_date', 'recommended_product_id',
       'recommended_product_name', 'campaign_budget', 'total_campaign_cost',
       'total_revenue_generated'],
      dtype='object')

In [86]:
merged_customer.columns

Index(['engagement_id', 'customer_id', 'campaign_id', 'channel', 'sent',
       'delivered', 'opened', 'clicked', 'engagement_date', 'engagement_time',
       'conversion_status', 'customer_segment_x', 'campaign_name',
       'campaign_type', 'start_date', 'end_date', 'recommended_product_id',
       'recommended_product_name', 'campaign_budget', 'total_campaign_cost',
       'total_revenue_generated', 'age', 'job', 'marital', 'education',
       'credit_default', 'customer_segment_y', 'region', 'income',
       'created_at', 'credit_card', 'personal_loan', 'mortgage',
       'savings_account', 'investment_product', 'auto_loan',
       'wealth_management'],
      dtype='object')

# EDA

In [87]:
# How people are engaged
engagement["channel"].value_counts()

channel
Email                       11698
Mobile App Notifications     8324
SMS                          7112
Direct Mail                  1787
Name: count, dtype: int64

In [88]:
# What type of campaigns are there
campaigns["campaign_type"].value_counts()

campaign_type
Email                       21
Mobile App Notifications    13
SMS                         13
Direct Mail                  3
Name: count, dtype: int64

In [89]:
# Relationship between clicked and conversion_status
pd.crosstab(engagement["clicked"], engagement["conversion_status"])

# Seems like it does not matter most of the time, there exist a lot of people who clicked but did not convert or did not click but convert

conversion_status,No,Yes
clicked,,
No,24197,2449
Yes,2069,206


In [90]:
# Relatonship between channel and conversion_status
chanxcon = pd.crosstab(engagement["channel"], engagement["conversion_status"])
chanxcon["rate"] = chanxcon["Yes"] / (chanxcon["Yes"] + chanxcon["No"]) * 100
chanxcon

# Seems like email is performing the best whereas direct mail is performing the worst

conversion_status,No,Yes,rate
channel,,,
Direct Mail,1665,122,6.827084
Email,10380,1318,11.266883
Mobile App Notifications,7638,686,8.241230
SMS,6583,529,7.438133


In [91]:
# Relationship between campaign_id and conversion_status
idxcon = pd.crosstab(engagement["campaign_id"], engagement["conversion_status"])
idxcon["rate"] = idxcon["Yes"] / (idxcon["Yes"] + idxcon["No"]) * 100
idxcon
# Seems like campaigns performed almost equally

conversion_status,No,Yes,rate
campaign_id,,,
1,421,52,10.993658
2,681,59,7.972973
3,833,113,11.945032
4,522,48,8.421053
5,763,66,7.961399
6,518,61,10.535406
7,451,42,8.519270
8,899,70,7.223942
9,750,58,7.178218


In [92]:
merged["customer_id"].value_counts().shape
# There are 9381 customers getting engaged in our dataset

(9381,)

In [93]:
# See the number of customers that were engaged
(merged["customer_id"].value_counts() > 1).sum()

# There are 7760 customers that were engaged at least twice.
# Potential to see if the bank recommending the same thing to this customer to see personalization


7760

In [94]:
# Examine how much the engagement match the customer segment
print(f'Number of engagement that matched the campaigns: {(merged_customer["customer_segment_x"] == merged_customer["customer_segment_y"]).sum()}')
print(f'Number of engagement in total: {engagement.shape[0]}')

Number of engagement that matched the campaigns: 6265
Number of engagement in total: 28921


In [95]:
m = merged_customer
((m["customer_segment_x"] == m["customer_segment_y"]) & (m["conversion_status"] == True)).sum()

# Erm this means the engagement that matches the customer segment is really ass
# Ig this continues the propaganda that their segmentation cmi

0

In [96]:
eng = ["sent", "delivered", "opened", "clicked", "conversion_status"]
for en in eng:
    print(merged[en].value_counts())

sent
Yes    28921
Name: count, dtype: int64
delivered
Yes    26188
No      2733
Name: count, dtype: int64
opened
No     22013
Yes     6908
Name: count, dtype: int64
clicked
No     26646
Yes     2275
Name: count, dtype: int64
conversion_status
No     26266
Yes     2655
Name: count, dtype: int64


# Feature Engineering

In [97]:
merged.columns

Index(['engagement_id', 'customer_id', 'campaign_id', 'channel', 'sent',
       'delivered', 'opened', 'clicked', 'engagement_date', 'engagement_time',
       'conversion_status', 'customer_segment', 'campaign_name',
       'campaign_type', 'start_date', 'end_date', 'recommended_product_id',
       'recommended_product_name', 'campaign_budget', 'total_campaign_cost',
       'total_revenue_generated'],
      dtype='object')

In [98]:
merged.groupby("campaign_id").agg(successful_conversions=("conversion_status", lambda x: (x == "Yes").sum()),
                                  successful_clicked = ("clicked", lambda x: (x == "Yes").sum()),
                                  successful_opened = ("opened", lambda x: (x == "Yes").sum()),
                                  successful_delivered = ("delivered", lambda x: (x == "Yes").sum()),
                                  successful_sent = ("sent", lambda x: (x == "Yes").sum()))


,successful_conversions,successful_clicked,successful_opened,successful_delivered,successful_sent
campaign_id,,,,,
1,52,32,102,463,473
2,59,83,246,681,740
3,113,50,221,930,946
4,48,49,144,559,570
5,66,66,245,754,829
6,61,52,153,570,579
7,42,35,92,487,493
8,70,100,202,960,969
9,58,69,153,804,808


In [99]:
campaigns["ROI"] = (campaigns["total_revenue_generated"] - campaigns["total_campaign_cost"]) / campaigns["total_campaign_cost"]

In [100]:
campaigns.loc[:, ["campaign_id", "ROI"]]

,campaign_id,ROI
0,1,0.333645
1,2,1.105318
2,3,1.869173
3,4,1.546498
4,5,1.614992
5,6,0.616011
6,7,0.891929
7,8,0.148466
8,9,0.331467
9,10,1.692490


In [ ]:
# To create cost per conversion rate, cost per engagement
# To figure out personalization metrics

